## Quaternionen statt Rotationsmatrizen

#### Einlesen eines *.txt - Files

In [19]:
from IPython.display import display
import pandas as pd

pd.set_option('display.max_rows', None)  # Zeigt alle Zeilen an
pd.set_option('display.max_columns', None)  # Zeigt alle Spalten an
pd.set_option('display.expand_frame_repr', False)  # Keine Breitenbeschränkung

# display(df)  # DataFrame anzeigen

In [ ]:
df['Time'][0]

In [45]:
import pandas as pd

# Beispiel: Lesen der Daten aus einer Datei
# Pfad zur Datei
file_path = r"C:\Users\bernh\Downloads\drive-download-20240908T150007Z-001\WitMotion New Software\WitMotion(V2024.8.12.4)\Record\2024-09-15\05-11-18-527\data_0.txt"
# "C:\Users\bernh\Downloads\drive-download-20240908T150007Z-001\WitMotion New Software\WitMotion(V2024.8.12.4)\Record\2024-09-15\05-11-18-527\data_0.txt"
# Daten in den DataFrame laden, mit Tabulator als Trennzeichen
df = pd.read_csv(file_path, sep='\t', index_col=False)


# Überprüfen der Spaltennamen nach dem Zurücksetzen des Index
# print("Spaltennamen nach Index-Zurücksetzung:", df.columns)

# Die ersten paar Zeilen nach der Anpassung anzeigen
print(df.head())

#_# Ersetzen von Kommas durch Punkte in spezifischen Spalten
#_df['Chip Time()'] = df['Chip Time()'].str.replace(',', '.').astype(float)
#_# ACCELERATION:
df['Acceleration X(g)'] = df['Acceleration X(g)'].str.replace(',', '.').astype(float)
df['Acceleration Y(g)'] = df['Acceleration Y(g)'].str.replace(',', '.').astype(float)
df['Acceleration Z(g)'] = df['Acceleration Z(g)'].str.replace(',', '.').astype(float)
#_# ANGELS
df['Angle X(°)'] = df['Angle X(°)'].str.replace(',', '.').astype(float)
df['Angle Y(°)'] = df['Angle Y(°)'].str.replace(',', '.').astype(float)
df['Angle Z(°)'] = df['Angle Z(°)'].str.replace(',', '.').astype(float)
# ANGULAR VELOSITY
df['Angular velocity X(°/s)'] = df['Angular velocity X(°/s)'].str.replace(',', '.').astype(float)
df['Angular velocity Y(°/s)'] = df['Angular velocity Y(°/s)'].str.replace(',', '.').astype(float)
df['Angular velocity Z(°/s)'] = df['Angular velocity Z(°/s)'].str.replace(',', '.').astype(float)

# Alle numerischen Spalten mit Kommas in Punkten umwandeln
# df = df.replace({',': '.'}, regex=True).astype(float)

# print(df)

            Time Device name  Chip Time() Acceleration X(g) Acceleration Y(g) Acceleration Z(g) Angular velocity X(°/s) Angular velocity Y(°/s) Angular velocity Z(°/s) Angle X(°) Angle Y(°) Angle Z(°) Magnetic field X(ʯt) Magnetic field Y(ʯt) Magnetic field Z(ʯt) Temperature(℃)  Pressure(Pa) Height(m)  Longitude()  Latitude()  GPS ground speed(km)  Angle of course(°)  GPS height(m)  Number of satellites()  Position location()  Horizontal position()  Vertical position()  Quaternions 0()  Quaternions 1()  Quaternions 2()  Quaternions 3()
0   05:11:18.625       COM12          NaN            -0,021             0,017             1,008                   0,000                   0,000                   0,000      0,851      1,044      0,665               69,758               45,968               98,332          18,35         96787    386,73          NaN         NaN                   NaN                 NaN            NaN                     NaN                  NaN                    NaN      

In [51]:
print(df.columns)

Index(['Time', 'Device name', 'Chip Time()', 'Acceleration X(g)',
       'Acceleration Y(g)', 'Acceleration Z(g)', 'Angular velocity X(°/s)',
       'Angular velocity Y(°/s)', 'Angular velocity Z(°/s)', 'Angle X(°)',
       'Angle Y(°)', 'Angle Z(°)', 'Magnetic field X(ʯt)',
       'Magnetic field Y(ʯt)', 'Magnetic field Z(ʯt)', 'Temperature(℃)',
       'Pressure(Pa)', 'Height(m)', 'Longitude()', 'Latitude()',
       'GPS ground speed(km)', 'Angle of course(°)', 'GPS height(m)',
       'Number of satellites()', 'Position location()',
       'Horizontal position()', 'Vertical position()', 'Quaternions 0()',
       'Quaternions 1()', 'Quaternions 2()', 'Quaternions 3()', 'Delta_t',
       'Position X', 'Position Y', 'Position Z'],
      dtype='object')


### eigentliche Zeit-Differenz-Berechnung

In [47]:
import numpy as np
from datetime import datetime

time = df['Time'].values  # Zeitspalte falls vorhanden

# Dein NumPy-Array mit Zeitstempeln als Strings
# time = np.array([' 18:06:05.104', ' 18:06:05.247', ' 18:06:05.304', ' 18:06:05.403'])

# Entferne führende und nachfolgende Leerzeichen aus der 'Time'-Spalte
df['Time'] = df['Time'].str.strip()
 
# Konvertiere die 'Time'-Spalte in das Datetime-Format
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S.%f')

# Zeitdifferenzen berechnen und in einer neuen Spalte speichern
df['Delta_t'] = df['Time'].diff().dt.total_seconds()
 
# print(df['Delta_t'])

### Rotationsmatrix 

### Quaternionen

In [49]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R

# Beispiel für deinen DataFrame (in der Praxis durch deine Daten ersetzen)
# df = pd.read_csv('deine_daten.csv')

# Initiale Geschwindigkeit und Position setzen
velocity = np.array([0.0, 0.0, 0.0])
position = np.array([0.0, 0.0, 0.0])

# Zeitschritt (delta_t), basierend auf deiner Datenfrequenz
delta_t = 0.01  # Beispiel: 10 ms

# Schleife über alle Zeilen des DataFrames
for index, row in df.iterrows():
    # Winkel in Radiant umwandeln
    roll = np.radians(row['Angle X(°)'])
    pitch = np.radians(row['Angle Y(°)'])
    yaw = np.radians(row['Angle Z(°)'])

    # Quaternion aus Roll, Pitch, Yaw (X, Y, Z-Achsenrotation)
    rotation = R.from_euler('xyz', [roll, pitch, yaw])

    # Erdbeschleunigung in m/s²
    g = 9.81


    # Beschleunigungsdaten in g aus dem DataFrame (in m/s² umrechnen: 1g = 9.81 m/s²)
    acc_sensor = np.array([row['Acceleration X(g)'] * g,
                           row['Acceleration Y(g)'] * g,
                           row['Acceleration Z(g)'] * g-g])

    # Beschleunigung in Weltkoordinaten transformieren
    acc_world = rotation.apply(acc_sensor)

    # Geschwindigkeit berechnen (Integration der Beschleunigung)
    velocity += acc_world * delta_t

    # Position berechnen (Integration der Geschwindigkeit)
    position += velocity * delta_t

    # Optional: Position in eine neue Spalte im DataFrame schreiben
    df.at[index, 'Position X'] = position[0]
    df.at[index, 'Position Y'] = position[1]
    df.at[index, 'Position Z'] = position[2]

# Am Ende kannst du den DataFrame speichern oder die Positionen weiterverarbeiten
df.to_csv('positions_output.csv', index=False, decimal=',')


## Anpassung mit Hochpassfilter

In [40]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R

# Beispiel für das Laden des DataFrames (ersetze dies durch deine tatsächlichen Daten)
# df = pd.read_csv('positions_output.csv', decimal=',')

# Initiale Geschwindigkeit und Position setzen
velocity = np.array([0.0, 0.0, 0.0])
position = np.array([0.0, 0.0, 0.0])

# Zeitschritt (delta_t), basierend auf deiner Datenfrequenz
delta_t = 0.01  # Beispiel: 10 ms

# Erdbeschleunigung in m/s²
g = 9.81

# Hochpassfilter Parameter
alpha = 0.9  # Filterfaktor, je näher bei 1, desto stärker die Glättung
prev_acc_filtered = np.array([0.0, 0.0, 0.0])

# Schleife über alle Zeilen des DataFrames
for index, row in df.iterrows():
    # Winkel in Radiant umwandeln
    roll = np.radians(row['Angle X(°)'])
    pitch = np.radians(row['Angle Y(°)'])
    yaw = np.radians(row['Angle Z(°)'])

    # Quaternion aus Roll, Pitch, Yaw (X, Y, Z-Achsenrotation)
    rotation = R.from_euler('xyz', [roll, pitch, yaw])

    # Beschleunigungsdaten korrigieren
    acc_sensor = np.array([
        row['Acceleration X(g)'] * g,
        row['Acceleration Y(g)'] * g,
        row['Acceleration Z(g)'] * g - g  # Erdbeschleunigung korrigieren
    ])

    # Hochpassfilter anwenden
    acc_filtered = alpha * (prev_acc_filtered + acc_sensor - prev_acc_filtered)
    prev_acc_filtered = acc_filtered

    # Beschleunigung in Weltkoordinaten transformieren
    acc_world = rotation.apply(acc_filtered)

    # Geschwindigkeit berechnen (Integration der Beschleunigung)
    velocity += acc_world * delta_t

    # Position berechnen (Integration der Geschwindigkeit)
    position += velocity * delta_t

    # Optional: Position in eine neue Spalte im DataFrame schreiben
    df.at[index, 'Position X'] = position[0]
    df.at[index, 'Position Y'] = position[1]
    df.at[index, 'Position Z'] = position[2]

# Speichern des DataFrames mit einem Komma als Dezimaltrennzeichen
df.to_csv('positions_output2.csv', index=False, decimal=',')


### Graphische Ausgabe

In [50]:
import pandas as pd
import plotly.graph_objects as go

# Beispiel für das Laden des DataFrames (ersetze dies durch deine tatsächlichen Daten)
# df = pd.read_csv('positions_output.csv', decimal=',')

# Erstelle ein Linien-Diagramm für die Positionen X, Y und Z
fig = go.Figure()

# Füge die Positionen für X, Y und Z als separate Linien hinzu
fig.add_trace(go.Scatter(x=df.index, y=df['Position X'], mode='lines', name='Position X'))
fig.add_trace(go.Scatter(x=df.index, y=df['Position Y'], mode='lines', name='Position Y'))
fig.add_trace(go.Scatter(x=df.index, y=df['Position Z'], mode='lines', name='Position Z'))

# Diagramm-Titel und Achsenbeschriftungen hinzufügen
fig.update_layout(
    title='Positionen im Raum',
    xaxis_title='Index',
    yaxis_title='Position (m)',
    legend_title='Position',
)

# Diagramm anzeigen
fig.show()